In [ ]:
%store -r

In [ ]:
!pip install confluent-kafka==2.13.0

In [ ]:
import json
import time
from datetime import datetime, timezone
from confluent_kafka import Producer
import pandas as pd

In [ ]:
def delivery_callback(err, msg):
    """Callback function to handle delivery reports"""
    if err:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

def put_to_topic(kafka_bootstrap_servers, topic_name, ticker, amount, timestamp):
    """Send message to Kafka topic using confluent-kafka Producer"""

    producer_config = {
        'bootstrap.servers': kafka_bootstrap_servers,
        'client.id': 'bedrock-stream-ingest-producer'
    }

    producer = Producer(producer_config)

    payload = {
        'ticker': ticker,
        'price': amount,
        'timestamp': timestamp
    }

    data = json.dumps(payload)
    encoded_message = data.encode('utf-8')

    print(f'Sending ticker data: {ticker}...')

    try:
        producer.produce(
            topic=topic_name,
            value=encoded_message,
            callback=delivery_callback
        )
        producer.flush()
        return True
    except Exception as e:
        print(f'Error producing message: {e}')
        return False

In [ ]:
df = pd.read_csv('TestData.csv')

start_test_time = time.time()
print(datetime.fromtimestamp(start_test_time, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))

df = df.reset_index()

for index, row in df.iterrows():
    put_to_topic(BootstrapBrokerString, KafkaTopic, row['ticker'], row['price'], time.time())

end_test_time = time.time()
print(datetime.fromtimestamp(end_test_time, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))